In [ ]:
import rospy
import actionlib
from geometry_msgs.msg import PoseStamped
from assignment_2_2023.msg import PlanningAction, PlanningGoal,PlanningActionResult, PositionVelocity
from assignment_2_2023.msg import bar
from nav_msgs.msg import Odometry
import ipywidgets as widgets
import matplotlib.pyplot as plt
import rospy
import tf
from turtlesim.msg import Pose
from tf.transformations import quaternion_matrix
import numpy as np
from matplotlib.animation import FuncAnimation
from nav_msgs.msg import Odometry

In [ ]:
# Create the publisher objects
pub_PosVel = rospy.Publisher('PosVel', PositionVelocity, queue_size=10)
pub_goal = rospy.Publisher('goal_topic', PlanningGoal, queue_size=10)


goal = None

goals_reached = 0
goals_canc = 0

# Initialize the node
rospy.init_node('Actionlib_Client', anonymous=True)

# Create an Actionlib client
client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)

In [ ]:
class Visualiser:
	def __init__(self):
		self.fig, self.ax = plt.subplots()
		self.ln, = plt.plot([], [], 'ro')
		self.x_data, self.y_data = [] , []

	def plot_init(self):
		self.ax.set_xlim(10, -10)
		self.ax.set_ylim(10, -10)
		return self.ln,

	def bar_init(self):
		x = ['Reached', 'Cancelled']
		y = [10, 20]
		self.ax.bar(x, y)
		return self.ax,
		
	def odom_callback(self, msg):
		self.y_data.append(msg.pose.pose.position.y)
		self.x_data.append(msg.pose.pose.position.x)

	def goal_callback(self, msg):
		self.y_data.append(msg.target_pose.pose.position.y)
		self.x_data.append(msg.target_pose.pose.position.x)

	def update_plot(self, frame):	
		self.ln.set_data(self.x_data, self.y_data)
		return self.ln,

	def update_bar(self, frame):
		self.ax.clear()
		x = ['Reached', 'Cancelled']
		y = [goals_reached, goals_canc]
		self.ax.bar(x, y, color = ['green', 'red'])
		return self.ax,

In [ ]:
def PosVel_Callback(data):

    """
    Callback function for the subscriber to the topic /odom.
    It creates a custom message with the position and velocity of the robot according to
    the custom message PositionVelocity.msg and publishes it on the topic /PosVel using 
    the publisher pub_PosVel.

    inputs: data (Odometry)
    """
    
    # Aquring Position and Velocity
    position_velocity = PositionVelocity()
    position_velocity.x = data.pose.pose.position.x
    position_velocity.y = data.pose.pose.position.y
    position_velocity.vel_x = data.twist.twist.linear.x
    position_velocity.vel_z = data.twist.twist.angular.z

    # Publish the custom message
    pub_PosVel.publish(position_velocity)

In [ ]:
def result_callback(msg):

    global goals_reached, goals_canc
    
    status = msg.status.status

    if status == 3:
        goals_reached += 1
        rospy.loginfo('Goal reached!')
    else:
        goals_canc += 1

In [ ]:
# %matplotlib widget

x = widgets.IntSlider(min=-8, max=8, value=0, description='x_cord')
y = widgets.IntSlider(min=-8, max=8, value=0, description='y_cord')

send_b = widgets.Button(
    description='Send goal',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

cancel_b = widgets.Button(
    description='cancel goal',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

button_box = widgets.HBox([send_b, cancel_b])

In [ ]:
def send_goal(b):

    goal = PlanningGoal()
    goal.target_pose.pose.position.x = x.value
    goal.target_pose.pose.position.y = y.value
    pub_goal.publish(goal)

    client.send_goal(goal)


def cancel_goal(b):
    client.cancel_goal()
    rospy.loginfo('Goal cancelled! Please send a new goal.')


send_b.on_click(send_goal)
cancel_b.on_click(cancel_goal)


In [ ]:
def main():
    
    display(x, y, button_box)
    
    # Subscribe to the topic /odom to publish the custom message about position and velocity
    rospy.Subscriber('/odom', Odometry, PosVel_Callback)

    # Subscribe to the topic /reaching_goal/result to read the status of the result
    rospy.Subscriber('/reaching_goal/result',PlanningActionResult, result_callback)


In [ ]:
%matplotlib widget

vis = Visualiser()
sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init, blit=True)
rospy.Subscriber('goal_topic', PlanningGoal, vis.goal_callback)
plt.show(block=True)

vis1 = Visualiser()
ani1 = FuncAnimation(vis1.fig, vis1.update_bar, init_func=vis1.bar_init, blit=True)
plt.show(block=True)

if __name__ == '__main__':
    main()
